In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
from tensorflow.keras.optimizers import Optimizer
import keras
import math
import graphviz 
import sklearn
import copy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
class DNMFLoss(Loss):
    def call(self, y_true, y_pred):
                                
        return tf.reduce_mean(tf.math.square(y_pred - y_true), axis = -1) #Mean Square Error for now.

class DNMFOptimizer(Optimizer): #Preparing working space.
    
    def build(self, var_list):
        super().build(var_list)
    def update_step(self):
        pass
    def get_config(self):
        config = super().get_config()
        
        return config
    
    def __call__(self, loss, var_list, tape = None):
        minimize(loss = loss, var_list = var_list, tape = tape)
    
class DNMF(tf.Module): #It's just a Multilayer Perceptron for now.
    def __init__(self, input_size, sizes, name = None):
        super().__init__(name = name)
        self.layers = []
        with self.name_scope:
            for size in sizes:
                self.layers.append(Dense(input_dim = input_size, output_size = size))
                input_size = size
    @tf.Module.with_name_scope
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
       
input_size = 1
sizes = []
loss_object = DNMFLoss()
optimizer = DNMFOptimizer(input_size, sizes)
        
def loss(model, x, y, training):
    y_ = model(x, training = training)
    
    return loss_object(y_true = y, y_pred = y_)
        
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training = True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)
        
def optimization_step():
    loss_value, grads = grad(model, features, labels)
    print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()))
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss(model, features, labels, training = True).numpy()))